# Building a Local Agent with Strands Agents and Ollama Model

This notebook demonstrates how to create a personal agent using Strands Agent and Ollama. The agent will be capable of performing various local tasks such as file operations, web searches, and system commands.

## What is Ollama?

[Ollama](https://ollama.com/) is an open-source framework that allows you to run large language models (LLMs) locally on your machine. It provides a simple API for interacting with these models, making it ideal for privacy-focused applications where you don't want to send data to external services.

Key benefits of Ollama:
- **Privacy**: All processing happens locally on your machine
- **No API costs**: Free to use as much as you want
- **Offline capability**: Works without internet connection
- **Customization**: Can be fine-tuned for specific use 


## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |Ollama Model - to create a file operations agent   |
|Agent Structure     |Single agent architecture                          |


</div>


### Agent Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [5]:
!pip install -r requirements.txt

## Setup and Installation

Before running this notebook, make sure you have:

1. Install Ollama: Follow instructions at [https://ollama.com/download](https://ollama.com/download)
2. Start the Ollama server: `ollama serve`
3. Downloaded a model with Ollama: `ollama pull llama3.2:1b`

Refer to the [documentation](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) for detailed instructions.

In this notebook, we will download Ollama for the linux distribution for compatibility with SageMaker Studio. This is done for code execution during AWS lead workshops on Workshop Studio. If you are running this code locally, you should adjust the steps to download Ollama to your current enviroment.

In [1]:
# this will work on linux computers
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
################################################################          89.1%curl: (56) OpenSSL SSL_read: Connection reset by peer, errno 104


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [2]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

<Popen: returncode: None args: ['ollama', 'serve']>

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [3]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 492 KB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   8% ▕█                 ▏ 107 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  18% ▕███               ▏ 243 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  24% ▕████              ▏ 315 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  34% ▕██████            ▏ 447 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  42% ▕███████           ▏ 548 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  45% ▕████████   

In [6]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [7]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

✅ Ollama is running. Available models:
- llama3.2:1b


## Defining Custom Tools

Tools are functions that the agent can use to interact with the environment. Below, we define several useful tools for our personal agent.

In [8]:
# File Operation Tools


@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        with open(file_path, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Creating the Ollama-powered Agent

Now we'll create our agent using the Ollama model and the tools we defined above.

Note: You can add more tools like `execute_commands`, `search_files` etc.

In [9]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:1b"  # You can change this to any model you have pulled with Ollama.
)

#### Configure the Ollama model
Make sure your Ollama service is running at http://localhost:11434 and your `model_id` is in the list of Ollama models printed above.

In [10]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    params={
        "max_tokens": 4096,  # Adjust based on your model's capabilities
        "temperature": 0.7,  # Lower for more deterministic responses, higher for more creative
        "top_p": 0.9,  # Nucleus sampling parameter
        "stream": True,  # Enable streaming responses
    },
)

# Create the agent
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## Testing the Agent

Let's test our agent with some example tasks.

In [11]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)


Tool #1: file_read
The error message indicates that the program is having trouble decoding a specific byte in the file. This could be due to the file not being in a format that the program can read, or there might be an issue with the file itself.

Let's try to summarize the file instead:

```
file_summarized = open("sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf", 'r').read()
print(file_summarized)
```

This code will attempt to read the entire file into memory and then print it out. This might be more reliable if the file is too large for your program to handle.

Alternatively, you could use a library like `pdfplumber` which can parse PDF files and extract text content:

```
import pdfplumber

with pdfplumber.open("sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf") as pdf:
    print(pdf.extract_text())
```

This code will open the file, extract its text content, and then print it out.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The error message indicates that the program is having trouble decoding a specific byte in the file. This could be due to the file not being in a format that the program can read, or there might be an issue with the file itself.\n\nLet\'s try to summarize the file instead:\n\n```\nfile_summarized = open("sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf", \'r\').read()\nprint(file_summarized)\n```\n\nThis code will attempt to read the entire file into memory and then print it out. This might be more reliable if the file is too large for your program to handle.\n\nAlternatively, you could use a library like `pdfplumber` which can parse PDF files and extract text content:\n\n```\nimport pdfplumber\n\nwith pdfplumber.open("sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf") as pdf:\n    print(pdf.extract_text())\n```\n\nThis code will open the file, extract its text content, and then print it

In [12]:
# Example 2: List files in the current directory
response = local_agent("Show me the files in the current directory")


Tool #2: list_directory
The files in the current directory are:

1. `requirements.txt`
2. `owl.json`
3. `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf`

These files appear to be part of an Open Learning Lab (OLL) project, which is a collection of pre-trained models and tools for various tasks. The specific file mentioned in the original question is a shareholder letter from Amazon.com Inc. published in 2023.

In [13]:
# Example 3: Create a sample file
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'"
)

{"name": "file_write", "parameters": {"file_path": "sample.txt", "content": "This is a test file created by my Ollama agent.", "type": "object", "required": ["file_path", "content"], "properties": {"file_path": "str", "content": "str"}}

In [14]:
# Example 4: create a readme file after reading and understanding multiple files
response = local_agent("Create a readme.md for the current directory")


Tool #3: file_write
```
# file_writeREADME.md

This repository contains a simple Python tool to write files.

## Creating a File

To create a new file, use the `file_write` tool with the following command:
```bash
python file_write.py --content "This is a test file created by my Ollama agent."
```
The `--content` parameter specifies the content of the file. The resulting file will be saved in the current directory.

## Viewing File Contents

To view the contents of an existing file, use the following command:
```bash
python file_write.py --file_path ./sample.txt
```
This will display the text of the `./sample.txt` file.

## Notes

* Make sure to run this script from the root directory of your project.
* The `--content` parameter is required for creating a new file, but optional when viewing an existing one.
```python
import file_write
file_write.py --content "This is a test file created by my Ollama agent."
```
```bash
python file_write.py --file_path ./sample.txt
```

## Conclusion

In this notebook, we've created a local personal agent using Stands and Ollama. The agent can perform file operations (read, write, append) and Summarize/Analyze text

This demonstrates the power of running AI models locally with Ollama, combined with the flexibility of strands' tool system. You can extend this agent by adding more tools or using different Ollama models based on your needs.

### Next Steps (Ideas)

- Try different Ollama models to see how they affect the agent's capabilities
- Add more complex tools like web search, email sending, or calendar integration
- Implement memory for the agent to remember past interactions
- Create a simple UI for interacting with your agent